In [1]:
# PHÁT BIỂU BÀI TOÁN
# Bài toán phân tích tình cảm (Sentiment Analysis) trên đánh giá sản phẩm Amazon
# Mục tiêu: Xây dựng mô hình phân loại có khả năng dự đoán xếp hạng sao (1-5) 
# dựa trên nội dung văn bản của đánh giá người dùng
# Dữ liệu: Bộ dữ liệu đánh giá Amazon về sản phẩm Avent Bottles

In [2]:
# NHẬP THƯ VIỆN VÀ TẢI DỮ LIỆU
import pandas as pd  # Để xử lý và phân tích dữ liệu
import string  # Để thực hiện các phép toán với chuỗi
import matplotlib.pyplot as plt  # Để trực quan hóa dữ liệu
import numpy as np  # Để thực hiện các phép toán số học
import seaborn as sns  # Để trực quan hóa dữ liệu thống kê
import math  # Để sử dụng các hàm toán học
import warnings  # Để xử lý các thông báo cảnh báo

In [4]:
# Tải dữ liệu thô từ tệp CSV
raw_data = pd.read_csv("amazon_reviews.csv")
print(raw_data)  # In dữ liệu thô để kiểm tra

                                 name  \
0    Philips Avent 3 Pack 9oz Bottles   
1    Philips Avent 3 Pack 9oz Bottles   
2    Philips Avent 3 Pack 9oz Bottles   
3    Philips Avent 3 Pack 9oz Bottles   
4    Philips Avent 3 Pack 9oz Bottles   
..                                ...   
186  Philips Avent 3 Pack 9oz Bottles   
187  Philips Avent 3 Pack 9oz Bottles   
188  Philips Avent 3 Pack 9oz Bottles   
189  Philips Avent 3 Pack 9oz Bottles   
190  Philips Avent 3 Pack 9oz Bottles   

                                                review  rating  count  \
0    I was recommended to use these bottles by a gi...       5    191   
1    If I had not been given a ton of Avent bottles...       2    191   
2    Leaks! Especially difficult to get a tight sea...       1    191   
3    I have been using the Avent bottle system for ...       5    191   
4    I used Avent bottles with my son when he was t...       5    191   
..                                                 ...     ...    ...

In [5]:
# THIẾT LẬP MÔI TRƯỜNG
# Tắt các cảnh báo để giữ cho đầu ra gọn gàng
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
np.random.seed(7)  # Đặt hạt giống ngẫu nhiên để đảm bảo kết quả có thể tái tạo

In [6]:
# Tải dữ liệu một cách tối ưu hơn
csv = "amazon_reviews.csv"
df = pd.read_csv(csv)
df.head(10)  # Hiển thị 10 hàng đầu tiên để kiểm tra dữ liệu

,name,review,rating,count,standev
0,Philips Avent 3 Pack 9oz Bottles,I was recommended to use these bottles by a gi...,5,191,1.620651
1,Philips Avent 3 Pack 9oz Bottles,If I had not been given a ton of Avent bottles...,2,191,1.620651
2,Philips Avent 3 Pack 9oz Bottles,Leaks! Especially difficult to get a tight sea...,1,191,1.620651
3,Philips Avent 3 Pack 9oz Bottles,I have been using the Avent bottle system for ...,5,191,1.620651
4,Philips Avent 3 Pack 9oz Bottles,I used Avent bottles with my son when he was t...,5,191,1.620651
5,Philips Avent 3 Pack 9oz Bottles,These bottles are simply the best out there. I...,5,191,1.620651
6,Philips Avent 3 Pack 9oz Bottles,I purchased these bottles for my older baby wh...,4,191,1.620651
7,Philips Avent 3 Pack 9oz Bottles,After reading the reviews of these bottles lea...,5,191,1.620651
8,Philips Avent 3 Pack 9oz Bottles,I really wanted to like these bottles because ...,1,191,1.620651
9,Philips Avent 3 Pack 9oz Bottles,"In the midst of all of my bottle testing, I wa...",5,191,1.620651


In [7]:
# KHÁM PHÁ BỘ DỮ LIỆU
# Tạo bản sao của dataframe để phân tích
data = df.copy()
data.describe()  # Tạo thống kê mô tả
data.info()  # In tóm tắt ngắn gọn của dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191 entries, 0 to 190
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     191 non-null    object 
 1   review   191 non-null    object 
 2   rating   191 non-null    int64  
 3   count    191 non-null    int64  
 4   standev  191 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 7.6+ KB


In [8]:
# Lưu ý: Xếp hạng của Avent Bottles phân bố ở các giá trị cực đoan
# Người dùng thường chỉ viết đánh giá khi rất hài lòng hoặc rất thất vọng

In [9]:
# XỬ LÝ GIÁ TRỊ THIẾU

In [10]:
from sklearn.model_selection import StratifiedShuffleSplit  # Để tạo phân chia tập huấn luyện/kiểm tra phân tầng

In [11]:
print("Before {}".format(len(data)))  # In kích thước bộ dữ liệu gốc
dataAfter = data.dropna(subset=["rating"])  # Loại bỏ các hàng có xếp hạng bị thiếu
print("After {}".format(len(dataAfter)))  # In kích thước bộ dữ liệu sau khi loại bỏ giá trị thiếu
dataAfter["rating"] = dataAfter["rating"].astype(int)  # Chuyển đổi xếp hạng thành số nguyên

Before 191
After 191


In [12]:
# CHIA DỮ LIỆU THÀNH TẬP HUẤN LUYỆN VÀ KIỂM TRA
# Sử dụng lấy mẫu phân tầng để duy trì cùng phân phối xếp hạng ở cả hai tập
split = StratifiedShuffleSplit(n_splits=5, test_size=0.25)  # Tạo 5 phân chia với 25% cho tập kiểm tra
for train_index, test_index in split.split(dataAfter, dataAfter["rating"]): 
    strat_train = dataAfter.reindex(train_index)  # Tạo tập dữ liệu huấn luyện
    strat_test = dataAfter.reindex(test_index)  # Tạo tập dữ liệu kiểm tra

In [13]:
# KIỂM TRA CÁC TẬP CON HUẤN LUYỆN VÀ KIỂM TRA
len(strat_train)  # Kiểm tra kích thước tập huấn luyện
strat_train["rating"].value_counts()/len(strat_train)  # Kiểm tra phân phối xếp hạng trong tập huấn luyện
len(strat_test)  # Kiểm tra kích thước tập kiểm tra
strat_test["rating"].value_counts()/len(strat_test)  # Kiểm tra phân phối xếp hạng trong tập kiểm tra

rating
5    0.354167
1    0.229167
2    0.166667
4    0.166667
3    0.083333
Name: count, dtype: float64

In [14]:
# Tạo bản sao của dữ liệu huấn luyện để phân tích đánh giá
reviews = strat_train.copy()
reviews.head(2)  # Hiển thị 2 hàng đầu tiên
reviews.info()  # In tóm tắt của dataframe đánh giá
data.describe()  # Tạo thống kê mô tả một lần nữa

<class 'pandas.core.frame.DataFrame'>
Index: 143 entries, 157 to 36
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     143 non-null    object 
 1   review   143 non-null    object 
 2   rating   143 non-null    int32  
 3   count    143 non-null    int64  
 4   standev  143 non-null    float64
dtypes: float64(1), int32(1), int64(1), object(2)
memory usage: 6.1+ KB


,rating,count,standev
count,191.000000,191.0,1.910000e+02
mean,3.204188,191.0,1.620651e+00
std,1.620651,0.0,2.226282e-16
min,1.000000,191.0,1.620651e+00
25%,2.000000,191.0,1.620651e+00
50%,4.000000,191.0,1.620651e+00
75%,5.000000,191.0,1.620651e+00
max,5.000000,191.0,1.620651e+00


In [15]:
# TẠO NHÃN TÌNH CẢM
# Định nghĩa hàm để chuyển đổi xếp hạng số thành danh mục tình cảm:
# - 4-5 sao = Tích cực
# - 3 sao = Trung tính
# - 1-2 sao = Tiêu cực
def sentiments(rating):
    if (rating == 5) or (rating == 4):
        return "Positive"
    elif rating == 3:
        return "Neutral"
    elif (rating == 2) or (rating == 1):
        return "Negative"

In [16]:
# Áp dụng gán nhãn tình cảm cho cả tập dữ liệu huấn luyện và kiểm tra
strat_train["Sentiment"] = strat_train["rating"].apply(sentiments)
strat_test["Sentiment"] = strat_test["rating"].apply(sentiments)
strat_train["Sentiment"][:20]  # Hiển thị 20 nhãn tình cảm đầu tiên để xác minh

157    Positive
68     Positive
179    Positive
54     Negative
82     Positive
119    Positive
7      Positive
28     Positive
117    Positive
103    Positive
96      Neutral
152    Positive
20     Negative
187    Positive
99     Positive
10     Negative
70      Neutral
107    Positive
165    Positive
181    Positive
Name: Sentiment, dtype: object

In [17]:
# CHUẨN BỊ DỮ LIỆU CHO MÔ HÌNH
# Trích xuất đặc trưng (X) và mục tiêu (y) cho cả tập huấn luyện và kiểm tra
X_train = strat_train["review"]  # Văn bản đánh giá cho huấn luyện
X_train_targetRating = strat_train["rating"]  # Xếp hạng số cho huấn luyện
X_test = strat_test["review"]  # Văn bản đánh giá cho kiểm tra
X_test_targetRating = strat_test["rating"]  # Xếp hạng số cho kiểm tra
print(len(X_train), len(X_test))  # In kích thước của tập huấn luyện và kiểm tra

143 48


In [18]:
# Xử lý giá trị thiếu trong đặc trưng và mục tiêu
X_train = X_train.fillna(' ')  # Thay thế NaN bằng chuỗi trống trong đánh giá huấn luyện
X_test = X_test.fillna(' ')  # Thay thế NaN bằng chuỗi trống trong đánh giá kiểm tra
X_train_targetRating = X_train_targetRating.fillna(' ')  # Thay thế NaN trong xếp hạng huấn luyện
X_test_targetRating = X_test_targetRating.fillna(' ')  # Thay thế NaN trong xếp hạng kiểm tra

In [20]:
# TIỀN XỬ LÝ VĂN BẢN VÀ TRÍCH XUẤT ĐẶC TRƯNG
from sklearn.feature_extraction.text import CountVectorizer  # Để chuyển đổi văn bản thành vector đếm từ

In [21]:
# Tạo CountVectorizer để chuyển đổi văn bản thành ma trận đếm từ
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)  # Chuyển đổi văn bản huấn luyện thành ma trận đếm
X_train_counts.shape  # Kiểm tra kích thước của ma trận đếm

(143, 1939)

In [29]:
# ÁP DỤNG BIẾN ĐỔI TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer  # Để biến đổi TF-IDF

# Biến đổi ma trận đếm thành biểu diễn TF (tần suất thuật ngữ)
# Lưu ý: use_idf=False có nghĩa là chỉ sử dụng tần suất thuật ngữ, không sử dụng tần suất tài liệu nghịch đảo
tfidf_transformer = TfidfTransformer(use_idf=False)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)  # Áp dụng biến đổi
X_train_tfidf.shape  # Kiểm tra kích thước của ma trận TF-IDF

(143, 1939)

In [30]:
# XÂY DỰNG VÀ HUẤN LUYỆN MÔ HÌNH
from sklearn.naive_bayes import MultinomialNB  # Bộ phân loại Naive Bayes cho mô hình đa thức
from sklearn.pipeline import Pipeline  # Để xây dựng đường ống các biến đổi và ước lượng
from sklearn.svm import LinearSVC  # Phân loại Support Vector tuyến tính

In [31]:
# Tạo đường ống cho phân loại văn bản sử dụng Linear SVC
# Đường ống bao gồm:
# 1. CountVectorizer: Chuyển đổi văn bản thành ma trận đếm từ
# 2. TfidfTransformer: Áp dụng trọng số TF-IDF
# 3. LinearSVC: Thuật toán Phân loại Support Vector tuyến tính
clf_linearSVC_pipe = Pipeline([("vect", CountVectorizer()), 
                               ("tfidf", TfidfTransformer()),
                               ("clf_linearSVC", LinearSVC())])

In [33]:
# Huấn luyện đường ống trên dữ liệu huấn luyện
clf_linearSVC_pipe.fit(X_train, X_train_targetRating)

C:\ProgramData\anaconda3a\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf_linearSVC', LinearSVC())])

In [34]:
# ĐÁNH GIÁ MÔ HÌNH
# Sử dụng mô hình đã huấn luyện để dự đoán xếp hạng cho tập kiểm tra
predictedLinearSVC = clf_linearSVC_pipe.predict(X_test)

In [35]:
# Tính toán độ chính xác bằng cách so sánh dự đoán với xếp hạng thực tế
# Điều này cho biết tỷ lệ đánh giá được phân loại chính xác
np.mean(predictedLinearSVC == X_test_targetRating)

0.5